# I2b2 dataset
Contents of the 2018 Task2 challenge:  
This dataset was created to identify Adverse Drug Events and Medication Extraction in EHRs. This challenge focused on three tasks:
- Identifying concepts: drug names, dosages, durations, etc.  
- Identifying relations: relation of drugs with ADE's and other entities given gold standard entities (generated by human annotators). 
- Running an end-to-end model that identifies relation of drugs with ADE's and other entittes on system predicted entitites.  

See documentation for more details.  

The training data is composed of individual notes (.txt extension) and corresponding individual annotation files (.ann extension).   
Annotation files contain tags (labeled with a leading 'T') and relations (labeled with a leading 'R'):
- For tags, the structure is: Tag_id, Tag_entity, Start_character_loc, End_character_loc  
- For relations, the structure is: Relation_id, Relation_entity, Arg1:Tag_id, Arg2:Tag_id  

In [1]:
import pandas as pd
import i2b2_evaluate as i2b2e
import glob, os
import sys, io
import re
import matplotlib.pyplot as plt
import numpy as np
import nltk
import statistics as stats
import itertools
from nltk.tokenize import sent_tokenize 
%matplotlib inline

In [2]:
#test_path = '/Users/valeriemeausoone/Documents/W266/github_repo/w266_final/data/i2b2/2018/training_20180910/training_20180910/100035.ann'
file_path = '/Users/valeriemeausoone/Documents/W266/github_repo/w266_final/data/i2b2/2018/training_20180910/training_20180910/'

In [3]:
os.chdir(file_path)

# Processing training data for BERT

## Sentence tokenizing

In [4]:
text_directory = sorted(glob.glob("*.txt"))
ann_directory = sorted(glob.glob("*.ann"))
list_files=[]
for file in text_directory:
    with open(file, 'rb') as f:
        text=f.read().decode("utf-8")
        list_files.append(text)

In [5]:
print(text_directory[0:10])
print(ann_directory[0:10])
print(len(list_files))

['100035.txt', '100039.txt', '100187.txt', '100229.txt', '100564.txt', '100579.txt', '100590.txt', '100677.txt', '100847.txt', '100883.txt']
['100035.ann', '100039.ann', '100187.ann', '100229.ann', '100564.ann', '100579.ann', '100590.ann', '100677.ann', '100847.ann', '100883.ann']
303


In [6]:
def sentence_tokenization(text):
    '''Splitting discharge summaries into sentences. Because discharge summaries are not consistently organized,
    extra processing is done to clean-up sentences and phrases. Chunks of texts are kept together to avoid splitting
    phrases too granularly'''
    #Using NLTK's sent_tokenize 
    sentence_tokens = sent_tokenize(text) 
    
    #Splititng paragraphs
    sentence_tokens2 = [paragraph for sentence in sentence_tokens for paragraph in sentence.split("\n\n\n")]
        
    #Removing sentences that are too short: only one dot (.) or a numerical bullet point (1., 2., 3.., ...10., etc.)    
    sentence_tokens3 = [sentence.strip() for sentence in sentence_tokens2 if (sentence != ".") or (re.match(r'\d*\.', sentence) is None)]

    #Cleaning up line breaks and replacing them with empty spaces
    sentence_tokens_clean = [sentence.replace('\n', ' ') for sentence in sentence_tokens3]
    
    #Saving results as dataframe 
    #sentences = pd.DataFrame(sentence_tokens_clean)
    #sentences = sentences.rename(columns={0:"sentences"})
    
    return sentence_tokens_clean

In [7]:
#100035.txt and 100039.txt
for file in list_files[0:2]:
    print(sentence_tokenization(file)[0])
    sentences = sentence_tokenization(file)

Admission Date:  [**2115-2-22**]              Discharge Date:   [**2115-3-19**]  Date of Birth:  [**2078-8-9**]             Sex:   M  Service: MEDICINE  Allergies: Vicodin  Attending:[**First Name3 (LF) 4891**] Chief Complaint: Post-cardiac arrest, asthma exacerbation  Major Surgical or Invasive Procedure: Intubation Removal of chest tubes placed at an outside hospital R CVL placement
Admission Date:  [**2174-4-18**]              Discharge Date:   [**2174-5-17**]  Date of Birth:  [**2135-11-15**]             Sex:   F  Service: MEDICINE  Allergies: Prochlorperazine / Heparin Agents  Attending:[**First Name3 (LF) 3918**] Chief Complaint: Abdominal Pain  Major Surgical or Invasive Procedure: Upper GI series with small bowel follow through Right heart catheterization IR guided paracentesis


## Linking with relations

In [8]:
def annotations_processing(file):
    '''This function processes the annotation files into dataframes (relation and concept). 
    It then combines these dataframes to create an enhanced relations dictionary'''
    
    #Reading the annotation file into a combined dataframe. 
    ann_df = pd.read_csv(file, sep="\t", header=None)
    ann_df = ann_df.rename(columns={0:"tag", 1:"description", 2:"text"})

    #Splitting concept entities and relations
    #Relations dataframe
    null_entries = pd.isnull(ann_df["text"])
    rf_df = ann_df[null_entries]
    rf_df = rf_df.rename(columns={'tag':"relation_id", 'description':"relation_description", 'text': 'relation_text'})
    #Cleaning up
    rf_df[['relation','arg1', 'arg2']] = rf_df['relation_description'].str.split(' ',expand=True)
    rf_df[['arg1_delete','arg1_keep']] = rf_df['arg1'].str.split(':',expand=True)
    rf_df[['arg2_delete','arg2_keep']] = rf_df['arg2'].str.split(':',expand=True)
    rf_df = rf_df.drop(columns=['relation_text', 'arg1', 'arg2', 'arg1_delete', 'arg2_delete'])
    rf_df = rf_df.rename(columns={'arg1_keep':"arg1", 'arg2_keep':"arg2"})
    
    #Concepts dataframe
    entries = pd.notnull(ann_df["text"])
    tag_df = ann_df[entries]
    tag_df = tag_df.rename(columns={'tag':"concept_id", 'description':"concept_description", 'text': 'concept_text'})

    #Combining relations and tags dataframes to create an enhanced relations dataframe
    rf_df = pd.merge(rf_df, tag_df, left_on = 'arg1', right_on='concept_id')
    rf_df = rf_df.rename(columns={'concept_id': 'arg1_id', 'concept_description':"arg1_description", 'concept_text':"arg1_text"})
    rf_df = pd.merge(rf_df, tag_df, left_on = 'arg2', right_on='concept_id')
    rf_df = rf_df.rename(columns={'concept_id': 'arg2_id', 'concept_description':"arg2_description", 'concept_text':"arg2_text"})
    rf_df = rf_df.drop(columns=['arg1_id', 'arg2_id'])

    #Creating a relations dictionary
    #Note that there could be "duplicate" relations that we will have to re-identify later. 
    dict_relation = {}
    for sentence in sentences: 
        for i in range(len(rf_df)):
            arg1 = rf_df['arg1_text'][i]
            arg2 = rf_df['arg2_text'][i]
            relation = rf_df['relation'][i]
            dict_relation[(arg1, arg2)] = relation
    return dict_relation

In [9]:
list_relations = [annotations_processing(file) for file in ann_directory]
len(list_relations)

303

## Compiling the dataframe

In [10]:
file_num=0
relation_sentences=[]
errors= 0
for file in list_files:
    
    #implementing sentence tokenization
    sentences = np.array(sentence_tokenization(file))

    #listing entities that make up relations
    list_entities = list(list_relations[file_num].keys())
    
    #looking for relation tags in sentences and pulling out sentences. 
    for e in list_entities:        
        new_e = e
        arg1_indices = np.where(np.char.find(sentences, new_e[0])>=0)[0]
        if arg1_indices.size==0: 
            new_e = list(new_e)
            new_e[0] = new_e[0].replace(' ', '')
            new_e = tuple(new_e)
            arg1_indices = np.where(np.char.find(sentences, new_e[0])>=0)[0]

        arg2_indices = np.where(np.char.find(sentences, new_e[1])>=0)[0]
        if arg2_indices.size==0: 
            new_e = list(new_e)
            new_e[1] = new_e[1].replace(' ', '')
            new_e = tuple(new_e)
            arg2_indices = np.where(np.char.find(sentences, new_e[1])>=0)[0]

        #extract where minimum. 
        combinations = [(i,j,abs(i-j)) for i,j in list(itertools.product(arg1_indices, arg2_indices))]
        try:
            min_distance = min(combinations, key = lambda t: t[2])[2]
        except ValueError:
            min_distance = "none"
        if min_distance != "none":
            min_combinations = [(t[0], t[1]) for t in combinations if t[2] == min_distance]
            for c in min_combinations:
                if c[0]==c[1]:
                    include_sentence = sentences[c[0]]
                    include_sentence = include_sentence.replace(new_e[0], ("SUB_B " + new_e[0] + " SUB_E"))
                    include_sentence = include_sentence.replace(new_e[1], ("OBJ_B " + new_e[1] + " OBJ_E"))
                    relation_sentences.append((new_e, list_relations[file_num][e], include_sentence))
                    sentences.tolist().pop(c[0])
                elif c[0]!=c[1]:
                    include_sentence = sentences[c[0]] + " " + sentences[c[1]]
                    include_sentence = include_sentence.replace(new_e[0], ("SUB_B " + new_e[0] + " SUB_E"))
                    include_sentence = include_sentence.replace(new_e[1], ("OBJ_B " + new_e[1] + " OBJ_E"))
                    relation_sentences.append((new_e, list_relations[file_num][e], include_sentence))
                    sentences.tolist().pop(c[0])
                    sentences.tolist().pop(c[1])
                    
    for s in range(len(sentences)):
        relation_sentences.append(("none", "no relation", sentences[s]))   
    #print("output length", len(relation_sentences))
    
    file_num+=1

In [11]:
print("Number of sentences", len(relation_sentences))

Number of sentences 76318


In [12]:
#Generating a dataframe
train_df = pd.DataFrame(relation_sentences)
train_df = train_df.rename(columns={0:"args", 1:"relation", 2: "sentence"})
train_df.head()

,args,relation,sentence
0,"(recurrent seizures, ativan)",Reason-Drug,He also may have SUB_B recurrent seizures SUB_...
1,"(IM, ativan)",Route-Drug,He also may have recurrent seizures which shou...
2,"(IV, ativan)",Route-Drug,He also may have recurrent seizures which shou...
3,"(25mg, Topiramate)",Strength-Drug,-patient will be on OBJ_B Topiramate OBJ_E SUB...
4,"(PO, Topiramate)",Route-Drug,-patient will be on OBJ_B Topiramate OBJ_E 25m...


In [13]:
train_df['relation'].value_counts()

no relation       40955
Strength-Drug      6781
Frequency-Drug     6484
Route-Drug         5925
Reason-Drug        5263
Form-Drug          4643
Dosage-Drug        4455
ADE-Drug           1167
Duration-Drug       645
Name: relation, dtype: int64

In [14]:
print("Percentage without a relation:", 40955*100/len(train_df))

Percentage without a relation: 53.66361801934013


In [15]:
label_list = train_df['relation'].unique()

label_to_ids_map =  {label: i for i, label in enumerate(label_list)}

def to_label_id(series):
    return label_to_ids_map[series]

train_df_bert = train_df.copy()
train_df_bert = pd.DataFrame({
    'id':range(len(train_df)),
    'label': train_df['relation'].apply(to_label_id),
    'alpha':['a']*train_df.shape[0],
    'text': train_df['sentence']
})

In [16]:
train_df_bert.head()

,id,label,alpha,text
0,0,0,a,He also may have SUB_B recurrent seizures SUB_...
1,1,1,a,He also may have recurrent seizures which shou...
2,2,1,a,He also may have recurrent seizures which shou...
3,3,2,a,-patient will be on OBJ_B Topiramate OBJ_E SUB...
4,4,1,a,-patient will be on OBJ_B Topiramate OBJ_E 25m...


In [17]:
train_df_bert.to_csv("i2b2_train_bert.csv")

# Processting test data for BERT

In [18]:
#test_path = '/Users/valeriemeausoone/Documents/W266/github_repo/w266_final/data/i2b2/2018/training_20180910/training_20180910/100035.ann'
file_path = '/Users/valeriemeausoone/Documents/W266/github_repo/w266_final/data/i2b2/2018/gold_standard_test/'
os.chdir(file_path)

## Sentence Tokenizing

In [19]:
text_directory = sorted(glob.glob("*.txt"))
ann_directory = sorted(glob.glob("*.ann"))
list_files=[]
for file in text_directory:
    with open(file, 'rb') as f:
        text=f.read().decode("utf-8")
        list_files.append(text)

In [20]:
list_relations = [annotations_processing(file) for file in ann_directory]
len(list_relations)

201

In [21]:
file_num=0
relation_sentences=[]
errors= 0
for file in list_files:
    
    #implementing sentence tokenization
    sentences = np.array(sentence_tokenization(file))

    #listing entities that make up relations
    list_entities = list(list_relations[file_num].keys())
    
    #looking for relation tags in sentences and pulling out sentences. 
    for e in list_entities:        
        new_e = e
        arg1_indices = np.where(np.char.find(sentences, new_e[0])>=0)[0]
        if arg1_indices.size==0: 
            new_e = list(new_e)
            new_e[0] = new_e[0].replace(' ', '')
            new_e = tuple(new_e)
            arg1_indices = np.where(np.char.find(sentences, new_e[0])>=0)[0]

        arg2_indices = np.where(np.char.find(sentences, new_e[1])>=0)[0]
        if arg2_indices.size==0: 
            new_e = list(new_e)
            new_e[1] = new_e[1].replace(' ', '')
            new_e = tuple(new_e)
            arg2_indices = np.where(np.char.find(sentences, new_e[1])>=0)[0]

        #extract where minimum. 
        combinations = [(i,j,abs(i-j)) for i,j in list(itertools.product(arg1_indices, arg2_indices))]
        try:
            min_distance = min(combinations, key = lambda t: t[2])[2]
        except ValueError:
            min_distance = "none"
        if min_distance != "none":
            min_combinations = [(t[0], t[1]) for t in combinations if t[2] == min_distance]
            for c in min_combinations:
                if c[0]==c[1]:
                    include_sentence = sentences[c[0]]
                    include_sentence = include_sentence.replace(new_e[0], ("SUB_B " + new_e[0] + " SUB_E"))
                    include_sentence = include_sentence.replace(new_e[1], ("OBJ_B " + new_e[1] + " OBJ_E"))
                    relation_sentences.append((new_e, list_relations[file_num][e], include_sentence))
                    sentences.tolist().pop(c[0])
                elif c[0]!=c[1]:
                    include_sentence = sentences[c[0]] + " " + sentences[c[1]]
                    include_sentence = include_sentence.replace(new_e[0], ("SUB_B " + new_e[0] + " SUB_E"))
                    include_sentence = include_sentence.replace(new_e[1], ("OBJ_B " + new_e[1] + " OBJ_E"))
                    relation_sentences.append((new_e, list_relations[file_num][e], include_sentence))
                    sentences.tolist().pop(c[0])
                    sentences.tolist().pop(c[1])
                    
    for s in range(len(sentences)):
        relation_sentences.append(("none", "no relation", sentences[s]))   
    #print("output length", len(relation_sentences))
    
    file_num+=1
    
print("Number of sentences", len(relation_sentences))

Number of sentences 50010


In [22]:
#Generating a dataframe
test_df = pd.DataFrame(relation_sentences)
test_df = test_df.rename(columns={0:"args", 1:"relation", 2: "sentence"})

test_df['relation'].value_counts()

no relation       27438
Strength-Drug      4274
Frequency-Drug     4090
Route-Drug         3723
Reason-Drug        3464
Form-Drug          3024
Dosage-Drug        2802
ADE-Drug            775
Duration-Drug       420
Name: relation, dtype: int64

In [23]:
print("Percentage without a relation:", 40955*100/len(test_df))

Percentage without a relation: 81.89362127574485


In [24]:
label_list = test_df['relation'].unique()

label_to_ids_map =  {label: i for i, label in enumerate(label_list)}

def to_label_id(series):
    return label_to_ids_map[series]

test_df_bert = test_df.copy()
test_df_bert = pd.DataFrame({
    'id':range(len(test_df)),
    'label': test_df['relation'].apply(to_label_id),
    'alpha':['a']*test_df.shape[0],
    'text': test_df['sentence']
})

test_df_bert.head()

,id,label,alpha,text
0,0,0,a,"MEDICATIONS: Lipitor, Tylenol with Codeine, D..."
1,1,1,a,tapered over SUB_B one week SUB_E and disconti...
2,2,2,a,She was started on prophylactic OBJ_B Oxacilli...
3,3,0,a,The patient's course in the Intensive Care Uni...
4,4,3,a,The patient's course in the Intensive Care Uni...


In [25]:
test_df_bert.to_csv("i2b2_test_bert.csv")